<a href="https://colab.research.google.com/github/ath0217/hello-github/blob/main/Lab_Session_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
sns.set_style("darkgrid")

In [ ]:
!pip list | grep imb

In [ ]:
!mkdir data

In [ ]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=1NnDVngq_4OlIeMHHAZ8miD_isbfF6B5V', # autoMPG6  https://drive.google.com/file/d/1NnDVngq_4OlIeMHHAZ8miD_isbfF6B5V/view?usp=sharing
        ]
outputs = ['autoMPG6.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

**Bias-Variance decomposition**

In [ ]:
!pip install --upgrade mlxtend

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.linear_model import LinearRegression 
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

In [ ]:
X= df[['Horse_power','Displacement']]
y = df['Mpg']

In [ ]:
pipe = Pipeline([('poly',PolynomialFeatures()),
                                  ('lr',LinearRegression())])
pipe

In [ ]:
from sklearn import set_config
set_config(display="diagram")

In [ ]:
pipe

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.50, shuffle=True,random_state=i)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pipe.predict(X_test)

In [ ]:
loss, bias, var = [],[],[]
range_ = range(1,5)
for i in range_:
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.50, shuffle=True,random_state=i)
  pipe = Pipeline([('poly',PolynomialFeatures(degree=i)),('lr',LinearRegression())])
  avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        pipe, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='mse',
        random_seed=i)
  loss.append(avg_expected_loss)
  bias.append(avg_bias)
  var.append(avg_var)
#plotting the results
sns.lineplot(x=range_, y=loss, color='tab:red')
sns.lineplot(x=range_, y=bias, color ='tab:blue')
sns.lineplot(x=range_, y=var, color='tab:orange')
plt.xlabel('Flexibility')
plt.legend(['Mse','Bias','Variance'])

In [ ]:
df['Mpg'].plot()

In [ ]:
X= df[['Weight','Acceleration']]
y = df['Mpg']

In [ ]:
loss, bias, var = [],[],[]
range_ = range(1,5)
for i in range_:
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.50,random_state=i)
  pipe = Pipeline([('poly',SplineTransformer(degree=i)),('lr',LinearRegression())])
  avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        pipe, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='mse',
        random_seed=i)
  loss.append(avg_expected_loss)
  bias.append(avg_bias)
  var.append(avg_var)
#plotting the results
sns.lineplot(x=range_, y=loss, color='tab:red')
sns.lineplot(x=range_, y=bias, color ='tab:blue')
sns.lineplot(x=range_, y=var, color='tab:orange')
plt.xlabel('Flexibility')
plt.legend(['Mse','Bias','Variance'])

**Validation set**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
X= df[['Horse_power']]
y = df['Mpg']
print(X.shape)

In [ ]:
%%time
range_ = range(1,11)
for i in range(10):
  mses=[]
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.50,random_state=i+1)
  for j in range_:    
    pipe = Pipeline([('poly',PolynomialFeatures(degree=j)),('lr',LinearRegression())])
    pipe.fit(X_train, y_train)  
    #plotting the results
    y_pred = pipe.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)
    mses.append(mse)
  sns.lineplot(x=range_, y=mses)
plt.xlabel('Flexibility')
plt.ylabel('$MSE_{Test}$')

**Cross-validation**

In [ ]:
from sklearn.model_selection import KFold,LeaveOneOut

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=714)
cv

In [ ]:
%%time
range_ = range(1,11)
mses=[]
# run 9 times with differet seed (i)
for i in range(9):  
  mses.append([])
  cv = KFold(n_splits=10, shuffle=True, random_state=i)
  # this is 10-fold CV
  for k, (train_index, test_index) in enumerate(cv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]    
    mses[i].append([]) 
    # this is running 10 different LR with degrees from 1 to 10
    for j in range_:               
      pipe = Pipeline([('poly',PolynomialFeatures(degree=j)),('lr',LinearRegression())])
      pipe.fit(X_train, y_train)  
      #plotting the results
      y_pred = pipe.predict(X_test)
      mse = mean_squared_error(y_test,y_pred)
      mses[i][k].append(mse)

In [ ]:
np.array(mses).shape

In [ ]:
for i in range(9):
  sns.lineplot(x=range_, y=np.array(mses[i]).mean(axis=0))
plt.ylim((16,28))
plt.xlabel('Flexibility')
plt.ylabel('$MSE_{Test}$')